In [1]:
from functools import partial
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
import itertools
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from getConfig  import getConfig
from getData    import getData
from selectSet import selectSet
from getSet import getSet
from preProcess import preProcess
from calcRMSE import calcRMSE

Using TensorFlow backend.


In [2]:
# Parameters for the network
def getParms():
    L1Size       = [18]
    L2Size       = [12]
    #L3Size       = [9]
    activation   = ["tanh"]
    batchSize    = [32]
    learningRate = [2e-3]
    initializer  = [keras.initializers.TruncatedNormal(mean=0.0, stddev=0.5)]
    loss         = ["mse"]
    return list(itertools.product(L1Size,
                                  L2Size,
                                  #L3Size,
                                  activation,
                                  batchSize,
                                  learningRate,
                                  initializer,
                                  loss))

In [3]:
def createVal(d):
    # Split Training into Train and Val
    valSize = int(d["trainX"].shape[0]*.2)
    trainSize = d["trainX"].shape[0] - valSize
    d["trainX"] = d["trainX"].head(trainSize)
    d["trainY"] = d["trainY"].head(trainSize)
    d["valX"] = d["trainX"].tail(valSize)
    d["valY"] = d["trainY"].tail(valSize)
    
    del d['testX']["unit"]   # Not needed for outlier detection
    return d

In [4]:
def buildLayers(parmDict, featureCount):
    Dense   = partial(keras.layers.Dense,\
                      kernel_initializer=parmDict["initializer"])
    Dropout = partial(keras.layers.Dropout)
    
    nn = tf.keras.Sequential()
    nn.add(Dense(parmDict["l1Size"], activation=parmDict["activation"]))
    nn.add(Dropout(0.25))
    nn.add(Dense(parmDict["l2Size"], activation=parmDict["activation"]))
    nn.add(Dropout(0.25))
    #nn.add(Dense(parmDict["l3Size"], activation=parmDict["activation"]))
    #nn.add(Dense(parmDict["l2Size"], activation=parmDict["activation"]))
    #nn.add(layers.Dense(parmDict["l2Size"], activation=parmDict["activation"]))
    #nn.add(layers.Dense(parmDict["l1Size"], activation=parmDict["activation"]))
    nn.add(Dense(featureCount, activation=parmDict["activation"]))
    return nn

In [5]:
def buildNetwork(parmDict, featureCount):
    nn = buildLayers(parmDict, featureCount)
    sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(loss='mean_squared_error', optimizer=sgd)
    nn.compile(optimizer=sgd, loss="mse")
    '''nn.compile(optimizer=tf.train.AdamOptimizer(parmDict["lr"]),\
               loss=parmDict["loss"],\
               metrics=['accuracy'])'''
    return nn

In [6]:
def fitNetwork(dataDict, parmDict, nn):
    TB        = keras.callbacks.TensorBoard(log_dir=config["TBdir"])
    #modelSave = keras.callbacks.ModelCheckpoint(config["modelDir"],
    #                                            save_weights_only=True,
    #                                            verbose=1)
    
    data = np.array(dataDict["trainX"])
    nn.fit(data, data,\
                  batch_size=parmDict["batchSize"],\
                  epochs=40,\
                  validation_split=0.15,\
                  verbose=0,\
                  shuffle=True,
                  callbacks=[TB])

In [7]:
def scoreNetwork(df, nn):
    data  = np.array(df)
    preds = nn.predict(x=data)
    rmse  = calcRMSE(df, preds)
    # rmse is a series: one value for each column. Return the avg across all columns
    return rmse.mean()

In [8]:
config = getConfig()
parms = getParms()

Set = selectSet()

train, test = getData(config)
train = getSet(train, Set)
test = getSet(test, Set)

dataDict = preProcess(train, test, config)
dataDict = createVal(dataDict)

featureCount = dataDict["trainX"].shape[1]

Enter a number from 1-41

Processing set  1
removing setting3
removing sensor1
removing sensor5
removing sensor10
removing sensor16
removing sensor18
removing sensor19


In [9]:
results = []
for x in parms:
    parmDict = {}                  # holds the hyperparameter combination for one run
    parmDict['l1Size']      = x[0]
    parmDict['l2Size']      = x[1]
    #parmDict['l3Size']      = x[2]
    parmDict['activation']  = x[2]
    parmDict['batchSize']   = x[3]
    parmDict['lr']          = x[4]
    parmDict['initializer'] = x[5]
    parmDict['loss']        = x[6]
    
    nn = buildNetwork(parmDict, featureCount)
    fitNetwork(dataDict, parmDict, nn)
    score = scoreNetwork(dataDict["testX"], nn)
    tup = (x, score)
    results.append(tup)

In [10]:
print("{:<35}{:<8}".format("Parms", "RMSE") )
for r in results:
    print("{:<35}{:.2f}".format(str(r[0]), r[1]))

Parms                              RMSE    
(18, 12, 'tanh', 32, 0.002, <tensorflow.python.keras.initializers.TruncatedNormal object at 0x7f79e77b2be0>, 'mse')0.38


##### Save the model when you're optimized

In [20]:
keras.models.save_model(nn, filepath="autoencoder.hdf5")

In [19]:
preds.shape

(13096, 18)